In [1]:
import pandas as pd
from typing import List, Dict, Any
from langchain.schema import Document

from VectorStoreDao import VectorStoreDao
from Embeddings import EmbeddingModelFactory


In [2]:
def concat_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    원본 11개 컬럼을 6개 간소화 컬럼으로 변환
    
    Args:
        df: 원본 데이터프레임
        
    Returns:
        간소화된 메타데이터가 추가된 데이터프레임
    """
    # 복사본 생성
    result_df = df.copy()
    
    # 간소화 컬럼 생성
    result_df['성별'] = df[['여성/가족', '남성', '성소수자']].max(axis=1)
    result_df['정체성'] = df[['인종/국적', '지역', '종교']].max(axis=1)
    result_df['연령'] = df['연령']
    result_df['기타'] = df[['기타 혐오', '개인지칭']].max(axis=1)
    result_df['욕설'] = df['악플/욕설']
    result_df['혐오없음'] = df['clean']
    
    print("컬럼 변환 완료:")
    print(f"- 성별: {result_df['성별'].sum()}개")
    print(f"- 정체성: {result_df['정체성'].sum()}개")
    print(f"- 연령: {result_df['연령'].sum()}개")
    print(f"- 기타: {result_df['기타'].sum()}개")
    print(f"- 욕설: {result_df['욕설'].sum()}개")
    print(f"- 혐오없음: {result_df['혐오없음'].sum()}개")
    
    return result_df

In [3]:
def create_documents_from_df(df: pd.DataFrame) -> List[Document]:
    """
    데이터프레임을 Document 객체 리스트로 변환
    
    Args:
        df: 간소화된 메타데이터가 포함된 데이터프레임
        
    Returns:
        Document 객체 리스트
    """
    documents = []
    
    for idx, row in df.iterrows():
        # 빈 문장 제외
        if pd.isna(row['문장']) or row['문장'].strip() == '':
            continue
            
        # Document 객체 생성
        doc = Document(
            page_content=str(row['문장']).strip(),
            metadata={
                "성별": int(row['성별']),
                "정체성": int(row['정체성']),
                "연령": int(row['연령']),
                "기타": int(row['기타']),
                "욕설": int(row['욕설']),
                "혐오없음": int(row['혐오없음']),
                "row_index": idx  # 원본 데이터 추적용
            }
        )
        documents.append(doc)
    
    print(f"Document 변환 완료: {len(documents)}개")
    return documents

In [4]:
def upload_vecstore(
    df: pd.DataFrame,
    persist_directory: str = "./hate_speech_vectorstore",
    embedding_provider: str = "openai",
    batch_size: int = 100,
    clear_existing: bool = False
) -> VectorStoreDao:
    """
    데이터프레임을 벡터스토어에 업로드
    
    Args:
        df: 혐오표현 데이터프레임
        persist_directory: 벡터스토어 저장 경로
        embedding_provider: 임베딩 모델 제공자 ('openai' 또는 'upstage')
        batch_size: 배치 처리 크기
        clear_existing: 기존 데이터 삭제 여부
        
    Returns:
        VectorStoreDao 인스턴스
    """
    print("=== 벡터스토어 업로드 시작 ===")
    
    # 1. 컬럼 간소화
    print("1. 컬럼 간소화 중...")
    processed_df = concat_columns(df)
    
    # 2. Document 변환
    print("2. Document 객체 변환 중...")
    documents = create_documents_from_df(processed_df)
    
    if len(documents) == 0:
        raise ValueError("변환된 문서가 없습니다. 데이터를 확인해주세요.")
    
    # 3. DAO 초기화
    print("3. VectorStoreDao 초기화 중...")
    embedding_model = EmbeddingModelFactory.create_embedding_model(embedding_provider)
    dao = VectorStoreDao(
        persist_directory=persist_directory,
        embedding_model=embedding_model,
        collection_name="hate_speech_collection"
    )
    
    # 4. 기존 데이터 처리
    if clear_existing:
        print("4. 기존 데이터 삭제 중...")
        try:
            dao.clear_collection()
        except:
            print("   기존 컬렉션이 없거나 삭제 실패 (정상)")
    
    # 5. 배치 업로드
    print(f"5. 벡터스토어 업로드 중... (배치 크기: {batch_size})")
    
    total_batches = (len(documents) + batch_size - 1) // batch_size
    
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        batch_num = (i // batch_size) + 1
        
        print(f"   배치 {batch_num}/{total_batches}: {len(batch)}개 문서 처리 중...")
        
        if i == 0:
            # 첫 번째 배치는 create_vector_store로 생성
            dao.create_vector_store(batch, force_recreate=True)
        else:
            # 나머지는 add_documents로 추가
            dao.add_documents(batch)
    
    # 6. 완료 및 정보 출력
    print("6. 업로드 완료!")
    # dao.persist()
    
    info = dao.get_collection_info()
    print(f"=== 업로드 결과 ===")
    print(f"총 문서 수: {info.get('document_count', 'N/A')}")
    print(f"저장 경로: {info.get('persist_directory', 'N/A')}")
    print(f"컬렉션명: {info.get('collection_name', 'N/A')}")
    
    return dao

In [5]:
def test_search(dao: VectorStoreDao, test_queries: List[str] = None) -> None:
    """
    벡터스토어 검색 테스트
    
    Args:
        dao: VectorStoreDao 인스턴스
        test_queries: 테스트할 쿼리 리스트
    """
    if test_queries is None:
        test_queries = [
            "여성을 비하하는 말",
            "욕설이 포함된 문장", 
            "깨끗한 일반 문장",
            "나이 관련 차별 표현"
        ]
    
    print("=== 검색 테스트 ===")
    
    for query in test_queries:
        print(f"\n쿼리: '{query}'")
        try:
            results = dao.retrieve_with_scores(query, k=3, retriever_type="basic")
            for i, (doc, score) in enumerate(results, 1):
                print(f"  {i}. {doc.page_content}...")
                print(f"     메타데이터: {doc.metadata}")
                print(f"     유사도 점수: {score}")
            # print(results)
                
        except Exception as e:
            print(f"  검색 오류: {e}")


In [7]:
df = pd.read_csv("korean_unsmile_dataset/unsmile_train_v1.0.tsv", sep ="\t")
print(f"데이터 크기: {df.shape}")
print(f"컬럼: {df.columns.tolist()}")

데이터 크기: (15005, 12)
컬럼: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭']


In [6]:
dao = VectorStoreDao(
        persist_directory="./hate_speech_vectorstore",
        embedding_model = EmbeddingModelFactory.create_embedding_model('upstage'),
        collection_name="hate_speech_collection"
    )
dao.create_vector_store()
test_search(dao, )

기존 벡터스토어 로드 완료
=== 검색 테스트 ===

쿼리: '여성을 비하하는 말'
리트리버 설정 완료: basic
VectorStore Retriever: Retrieved with scores
  1. 된장녀 시절부터 여성비하발언은 죄다 쓰고 맘충...
     메타데이터: {'혐오없음': 0, '성별': 1, '정체성': 0, '욕설': 0, '기타': 0, '연령': 0, 'row_index': 9543}
     유사도 점수: 1.2595394849777222
  2. 아니 페미욕한게 왜 여성혐오야?...
     메타데이터: {'기타': 0, '연령': 0, '정체성': 0, '욕설': 0, '성별': 1, 'row_index': 3041, '혐오없음': 0}
     유사도 점수: 1.2652454376220703
  3. 한국여자는 다 김치녀라는 말은 잘 하다가 한국여자는 다 메갈이라는 말은 곧뒈져도 안함 ㅋㅋㅋㅋㅋ 김치녀=김치좆이 못가져서 짜증냈던 여자...
     메타데이터: {'기타': 0, '연령': 0, '성별': 1, '혐오없음': 0, '욕설': 0, 'row_index': 2365, '정체성': 0}
     유사도 점수: 1.297667384147644

쿼리: '욕설이 포함된 문장'
리트리버 설정 완료: basic
VectorStore Retriever: Retrieved with scores
  1. 저 mc새끼가 존나 욕하는데...
     메타데이터: {'정체성': 0, '욕설': 1, '혐오없음': 0, '기타': 0, 'row_index': 8339, '연령': 0, '성별': 0}
     유사도 점수: 1.2279211282730103
  2. 무슨 욕했는지 말해주면 안되노?...
     메타데이터: {'정체성': 0, '욕설': 0, '연령': 0, '혐오없음': 1, '기타': 0, 'row_index': 14914, '성별': 0}
     유사도 점수: 1.2611539363861084
  3. 야이 시